In [ ]:
import numpy as np
from numpy.linalg import norm
import re
import razdel
import spacy
import pymorphy2
from collections import Counter
from tqdm.notebook import tqdm
from summa.summarizer import summarize
from itertools import combinations
import networkx as nx

In [ ]:
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
import nltk; nltk.download('punkt');
from rouge import Rouge

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving text.txt to text (2).txt


**Сравним работу трёх известных алгоритмов суммаризации текста на примере какой-либо новостной статьи.**

In [ ]:
for f in uploaded.keys():
    file = open(f, 'r', encoding='ср-1251')
    lines = file.read(6000)

In [ ]:
lines = lines.replace('\n\n', ' ')
lines

'По сообщениям Роскосмоса, российская космическая станция «Луна-25» перешла на нерасчетную орбиту и столкнулась с Луной, прекратив свое существование. Специалисты считают, что это очень большая потеря для отечественной космонавтики. По данным инсайдеров, случилось примерно следующее: станции был задан тормозной импульс для того, чтобы немного уменьшить периселений - лунный перигей (ближайшая к Луне точка орбиты ее искусственного спутника). Это было необходимо для приближения момента удачной посадки. Однако двигательная установка отработала импульс неправильно, похоже, что в полтора раза больше положенного. Если это действительно так, то станция могла выйти на незамкнутую орбиту снижения, то есть прямой наводкой на Луну... В миссии «Луна-25» были поставлены как научные, так и более практические задачи. Первые связаны с разработкой проблем происхождения и эволюции Луны, вторые - с возможностью освоения Луны в будущем. Главной задачей экспедиции была отработка технологии мягкой посадки во

## Метод Луна

In [ ]:
BAD_POS = ("PREP", "NPRO", "CONJ", "PRCL", "NUMR", "PRED", "INTJ", "PUNCT", "CCONJ", "ADP", "DET", "ADV")
spacy_model = spacy.load("ru_core_news_md")

def sentenize(text):
    return [s.text for s in razdel.sentenize(text)]


def tokenize_sentence(sentence):
    sentence = sentence.strip().replace("\xa0", "")
    tokens = [token.lemma_ for token in spacy_model(sentence) if token.pos_ not in BAD_POS]
    tokens = [token for token in tokens if len(token) > 2]
    return tokens

def tokenize_text(text):
    all_tokens = []
    for sentence in sentenize(text):
        all_tokens.extend(tokenize_sentence(sentence))
    return all_tokens

text = lines
sentences = sentenize(text)
print(tokenize_sentence(sentences[0]))
print(tokenize_text(text))

['сообщение', 'роскосмос', 'российский', 'космический', 'станция', 'луна-25', 'перейти', 'нерасчётный', 'орбита', 'столкнуться', 'луна', 'прекратить', 'существование']
['сообщение', 'роскосмос', 'российский', 'космический', 'станция', 'луна-25', 'перейти', 'нерасчётный', 'орбита', 'столкнуться', 'луна', 'прекратить', 'существование', 'специалист', 'считать', 'что', 'это', 'больший', 'потеря', 'отечественный', 'космонавтика', 'данным', 'инсайдер', 'случиться', 'следующий', 'станция', 'быть', 'задать', 'тормозной', 'импульс', 'того', 'чтобы', 'уменьшить', 'периселений', 'лунный', 'перигей', 'близкий', 'луна', 'точка', 'орбита', 'искусственный', 'спутник', 'это', 'было', 'необходимый', 'приближение', 'момент', 'удачный', 'посадка', 'двигательный', 'установка', 'отработать', 'импульс', 'похожий', 'что', 'полтора', 'раз', 'больше', 'положенный', 'если', 'это', 'станция', 'мочь', 'выйти', 'незамкнутый', 'орбита', 'снижение', 'быть', 'прямой', 'наводка', 'луна', 'миссия', 'луна-25', 'были', '

In [ ]:
class LuhnSummarizer:

    def __init__(
        self,
        significant_percentage = 0.4,
        min_token_freq = 2,
        max_gap_size = 4,
        verbose = False
    ):
        self.significant_percentage = significant_percentage
        self.min_token_freq = min_token_freq
        self.max_gap_size = max_gap_size
        self.chunk_ending_mask = [0] * self.max_gap_size
        self.verbose = verbose

    def __call__(self, text, target_sentences_count):

        all_significant_tokens = self._get_significant_tokens(text)
        if self.verbose:
            print("Значимые токены: ", all_significant_tokens)

        ratings = []
        for sentence_index, sentence in enumerate(sentenize(text)):

            sentence_rating = self._get_chunk_ratings(sentence, all_significant_tokens)
            if self.verbose:
                print("\tПРЕДЛОЖЕНИЕ. Значимость: {}, текст: {}".format(sentence_rating, sentence))
            ratings.append((sentence_rating, sentence_index))

        ratings.sort(reverse=True)

        ratings = ratings[:target_sentences_count]
        indices = [index for _, index in ratings]
        indices.sort()
        return " ".join([sentences[index] for index in indices])

    def _get_significant_tokens(self, text):

        tokens_counter = Counter(tokenize_text(text))
        significant_tokens_max_count = int(len(tokens_counter) * self.significant_percentage)
        significant_tokens = tokens_counter.most_common(significant_tokens_max_count)
        significant_tokens = {token for token, cnt in significant_tokens if cnt >= self.min_token_freq}
        return significant_tokens

    def _get_chunk_ratings(self, sentence, significant_tokens):

        tokens = tokenize_sentence(sentence)

        chunks, masks = [], []
        in_chunk = False
        for token in tokens:
            is_significant_token = token in significant_tokens

            if is_significant_token and not in_chunk:
                in_chunk = True
                masks.append([int(is_significant_token)])
                chunks.append([token])
            elif in_chunk:
                last_mask = masks[-1]
                last_mask.append(int(is_significant_token))
                last_chunk = chunks[-1]
                last_chunk.append(token)
            if not chunks:
                continue

            last_chunk_ending_mask = masks[-1][-self.max_gap_size:]
            if last_chunk_ending_mask == self.chunk_ending_mask:
                in_chunk = False

        ratings = []
        for chunk, mask in zip(chunks, masks):
            rating = self._get_chunk_rating(mask, chunk)
            ratings.append(rating)
        return ratings

    def _get_chunk_rating(self, original_mask, chunk):

        original_mask = "".join(map(str, original_mask))
        mask = original_mask.rstrip("0")

        end_index = original_mask.rfind("1") + 1
        chunk = chunk[:end_index]
        assert len(mask) == len(chunk)
        chunk = " ".join(chunk)

        words_count = len(mask)
        assert words_count > 0
        significant_words_count = mask.count("1")
        assert significant_words_count > 0

        rating = significant_words_count * significant_words_count / words_count
        if self.verbose:
            print("ПРОМЕЖУТОК. Значимость: {}, маска: {}, текст: {}".format(rating, mask, chunk))
        return rating

In [ ]:
luhn = LuhnSummarizer(verbose=False)
summary = luhn(text, 3)
print("Итоговый реферат: {}".format(summary))

Итоговый реферат: Вся космическая программа остановилась на запусках к станции «Мир», а потом МКС, на выводе спутников разных стран на орбиту Земли. За эти годы не было новых российских научных программ по освоению космоса, разработки новых космических кораблей для полетов в космос и к той же Луне. «Получается, что госкорпорация в нашей стране — это плавный переход от государственного управления к частному, - отмечал бывший глава Счетной палаты Сергей Степашин.


**LSA**


In [ ]:
def calc_method_score(records, predict_func, nrows=1000):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if nrows is not None and i >= nrows:
            break

        text = lines
        prediction = predict_func(text, summary)
        references.append(summary)
        predictions.append(prediction)

    calc_scores(references, predictions)

In [ ]:
def predict_lsa(text, summary, lsa_summarizer, tokenizer, summary_size=3):
    parser = PlaintextParser.from_string(text, tokenizer)
    predicted_summary = lsa_summarizer(parser.document, summary_size)
    predicted_summary = " ".join([str(s) for s in predicted_summary])
    return predicted_summary

lsa_summarizer = LsaSummarizer()
tokenizer = Tokenizer("russian")
calc_method_score(lines, lambda x, y: predict_lsa(x, y, lsa_summarizer, tokenizer))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Count: 1000
Ref: Вся космическая программа остановилась на запусках к станции «Мир», а потом МКС, на выводе спутников разных стран на орбиту Земли. За эти годы не было новых российских научных программ по освоению космоса, разработки новых космических кораблей для полетов в космос и к той же Луне. «Получается, что госкорпорация в нашей стране — это плавный переход от государственного управления к частному, - отмечал бывший глава Счетной палаты Сергей Степашин.
Hyp: По данным инсайдеров, случилось примерно следующее: станции был задан тормозной импульс для того, чтобы немного уменьшить периселений - лунный перигей (ближайшая к Луне точка орбиты ее искусственного спутника). Но пока ничего из этого не удалось... Катастрофа вдруг напомнила, что прошлый полет к Луне был аж в 1976 году. «Получается, что госкорпорация в нашей стране — это плавный переход от государственного управления к частному, - отмечал бывший глава Счетной палаты Сергей Степашин.
BLEU:  0.5584443411693834
ROUGE:  {'rouge-

**Summa**

In [ ]:
def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [ ]:
def calc_summa_score(records, summary_part=0.1, lower=True, nrows=1000):
    references = []
    predictions = []

    for i, record in tqdm(enumerate(records)):
        if i >= nrows:
            break

        references.append(summary)

        text = lines
        text = text if not lower else text.lower()
        predicted_summary = summarize(text, ratio=summary_part, language='russian').replace("\n", " ")
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

calc_summa_score(lines)

0it [00:00, ?it/s]

Count: 1000
Ref: Вся космическая программа остановилась на запусках к станции «Мир», а потом МКС, на выводе спутников разных стран на орбиту Земли. За эти годы не было новых российских научных программ по освоению космоса, разработки новых космических кораблей для полетов в космос и к той же Луне. «Получается, что госкорпорация в нашей стране — это плавный переход от государственного управления к частному, - отмечал бывший глава Счетной палаты Сергей Степашин.
Hyp: если когда-нибудь ее найдут, это освободит космические миссии от необходимости доставлять воду с земли, а в перспективе она может понадобиться в качестве сырья для создания ракетного топлива (с помощью тока воду можно расщепить на кислород и водород). за эти годы не было новых российских научных программ по освоению космоса, разработки новых космических кораблей для полетов в космос и к той же луне. за это время пара миллиардеров в сша организовала, развила и успешно использует свои собственные космические программы. в рф на